# PyAudio Implementation

* By: **Fernando Garcia** (fergarciadlc)
* Date: 17 - Feb - 2021
* Desc: Exploring system audio devices with PyAudio

## Testing functionalities for PyAudio

In [ ]:
import pyaudio

In [1]:
p = pyaudio.PyAudio()

def get_audio_devices(p):
    devs = []
    for i in range(p.get_device_count()):
        devs.append(p.get_device_info_by_index(i))
    
    input_dev = p.get_default_input_device_info()
    output_dev = p.get_default_output_device_info()
    
    return devs, input_dev, output_dev

devs, input_dev, output_dev = get_audio_devices(p)

In [3]:
input_dev

{'index': 1,
 'structVersion': 2,
 'name': 'Micrófono (Realtek High Definit',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [4]:
output_dev

{'index': 4,
 'structVersion': 2,
 'name': 'Altavoz/Auricular (Realtek High',
 'hostApi': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 2,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [22]:
# Audio Devices (run to see the whole list)
len(devs)
# devs

51

Notice that **input devices** have `'maxOutputChannels': 0`, whereas **output devices** have `'maxInputChannels': 0`

In [29]:
audio_device = {
    "in": "",
    "out": "",
    "devices": {
        "input_list": [],
        "output_list": []
    }
}   

In [16]:
p = pyaudio.PyAudio()

def get_audio_devices(p):
    audio_device = {
        "in": "",
        "out": "",
        "devices": {
            "input_list": [],
            "output_list": []
        }
    }
    
    input_list = []
    output_list = []
    for i in range(p.get_device_count()):
        
        device = p.get_device_info_by_index(i)
        
        if device["maxInputChannels"] > 0:
            input_list.append(device)
        
        if device["maxOutputChannels"] > 0:
            output_list.append(device)
    
    audio_device["in"] = p.get_default_input_device_info()
    audio_device["out"] = p.get_default_output_device_info()
    audio_device["devices"]["input_list"] = input_list
    audio_device["devices"]["output_list"] = output_list
    
    return audio_device

audio_device = get_audio_devices(p)

### Run the cell below to see the whole dictionary of audio devices (51 devices listed!!!)

In [28]:
import json 
print("audio_device =", json.dumps(audio_device, indent=2))

# Prevent redundancy
##  get_device_info_by_index , or
## get_device_info_by_host_api_device_index

**From PyAudio [documentation](https://people.csail.mit.edu/hubert/pyaudio/docs/#pyaudio.PyAudio.get_device_info_by_host_api_device_index)**

**`get_device_info_by_host_api_device_index(host_api_index, host_api_device_index)`**
        
    Return a dictionary containing the Device parameters for a given Host API’s n’th device. The keys of the dictionary mirror the data fields of PortAudio’s PaDeviceInfo structure.

    Parameters:	
    host_api_index – The Host API index number
    host_api_device_index – The n’th device of the host API
    Raises:	
    IOError – for invalid indices

    Return type:	
    dict

**`get_device_info_by_index(device_index)`**

    Return the Device parameters for device specified in device_index as a dictionary. The keys of the dictionary mirror the data fields of PortAudio’s PaDeviceInfo structure.

    Parameters:	device_index – The device index
    Raises:	IOError – Invalid device_index.
    Return type:	dict

In [32]:
p = pyaudio.PyAudio()

def get_audio_devices(p):
    audio_device = {
        "in": "",
        "out": "",
        "devices": {
            "input_list": [],
            "output_list": []
        }
    }
    
    input_list = []
    output_list = []
    
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')
    
    for i in range(numdevices):

        device = p.get_device_info_by_host_api_device_index(0, i) # HOSTAPI 0
        
        if device["maxInputChannels"] > 0:
            input_list.append(device)
        
        if device["maxOutputChannels"] > 0:
            output_list.append(device)
    
    audio_device["in"] = p.get_default_input_device_info()
    audio_device["out"] = p.get_default_output_device_info()
    audio_device["devices"]["input_list"] = input_list
    audio_device["devices"]["output_list"] = output_list
    
    return audio_device

audio_device = get_audio_devices(p)

### much better!

In [37]:
print("audio_device =", json.dumps(audio_device, indent=4))

audio_device = {
    "in": {
        "index": 1,
        "structVersion": 2,
        "name": "Micr\u00f3fono (Realtek High Definit",
        "hostApi": 0,
        "maxInputChannels": 2,
        "maxOutputChannels": 0,
        "defaultLowInputLatency": 0.09,
        "defaultLowOutputLatency": 0.09,
        "defaultHighInputLatency": 0.18,
        "defaultHighOutputLatency": 0.18,
        "defaultSampleRate": 44100.0
    },
    "out": {
        "index": 4,
        "structVersion": 2,
        "name": "Altavoz/Auricular (Realtek High",
        "hostApi": 0,
        "maxInputChannels": 0,
        "maxOutputChannels": 2,
        "defaultLowInputLatency": 0.09,
        "defaultLowOutputLatency": 0.09,
        "defaultHighInputLatency": 0.18,
        "defaultHighOutputLatency": 0.18,
        "defaultSampleRate": 44100.0
    },
    "devices": {
        "input_list": [
            {
                "index": 0,
                "structVersion": 2,
                "name": "Asignador de sonido Micro

### we can reassign the current input and output device from the list of devices

In [39]:
audio_device["in"] = audio_device["devices"]["input_list"][0]
audio_device["out"] = audio_device["devices"]["output_list"][0]
print("audio_device =", json.dumps(audio_device, indent=4))

audio_device = {
    "in": {
        "index": 0,
        "structVersion": 2,
        "name": "Asignador de sonido Microsoft - Input",
        "hostApi": 0,
        "maxInputChannels": 2,
        "maxOutputChannels": 0,
        "defaultLowInputLatency": 0.09,
        "defaultLowOutputLatency": 0.09,
        "defaultHighInputLatency": 0.18,
        "defaultHighOutputLatency": 0.18,
        "defaultSampleRate": 44100.0
    },
    "out": {
        "index": 3,
        "structVersion": 2,
        "name": "Asignador de sonido Microsoft - Output",
        "hostApi": 0,
        "maxInputChannels": 0,
        "maxOutputChannels": 2,
        "defaultLowInputLatency": 0.09,
        "defaultLowOutputLatency": 0.09,
        "defaultHighInputLatency": 0.18,
        "defaultHighOutputLatency": 0.18,
        "defaultSampleRate": 44100.0
    },
    "devices": {
        "input_list": [
            {
                "index": 0,
                "structVersion": 2,
                "name": "Asignador de soni

# Testing...

## SoundDevice

In [8]:
import sounddevice as sd
sd.query_devices()
list(sd.query_devices())

   0 Asignador de sonido Microsoft - Input, MME (2 in, 0 out)
>  1 Mic in at front panel (black) (, MME (2 in, 0 out)
   2 Micrófono (Realtek High Definit, MME (2 in, 0 out)
   3 Mezcla estéreo (Realtek High De, MME (2 in, 0 out)
   4 Asignador de sonido Microsoft - Output, MME (0 in, 2 out)
<  5 Altavoz/Auricular (Realtek High, MME (0 in, 2 out)
   6 1 - NS19D220MX16A (AMD High Def, MME (0 in, 2 out)
   7 Controlador primario de captura de sonido, Windows DirectSound (2 in, 0 out)
   8 Mic in at front panel (black) (Realtek High Definition Audio), Windows DirectSound (2 in, 0 out)
   9 Micrófono (Realtek High Definition Audio), Windows DirectSound (2 in, 0 out)
  10 Mezcla estéreo (Realtek High Definition Audio), Windows DirectSound (2 in, 0 out)
  11 Controlador primario de sonido, Windows DirectSound (0 in, 2 out)
  12 Altavoz/Auricular (Realtek High Definition Audio), Windows DirectSound (0 in, 2 out)
  13 1 - NS19D220MX16A (AMD High Definition Audio Device), Windows DirectSound (0

In [38]:
import pyaudio
# https://gist.github.com/mansam/9332445
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
#for each audio device, determine if is an input or an output and add it to the appropriate list and dictionary
for i in range (0,numdevices):
        if p.get_device_info_by_host_api_device_index(0,i).get('maxInputChannels')>0:
                print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

        if p.get_device_info_by_host_api_device_index(0,i).get('maxOutputChannels')>0:
                print("Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

devinfo = p.get_device_info_by_index(1)
print("Selected device is ",devinfo.get('name'))
if p.is_format_supported(44100.0,  # Sample rate
                         input_device=devinfo["index"],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
      print('Yay!')
p.terminate()

Input Device id  0  -  Asignador de sonido Microsoft - Input
Input Device id  1  -  Micrófono (Realtek High Definit
Input Device id  2  -  Mezcla estéreo (Realtek High De
Output Device id  3  -  Asignador de sonido Microsoft - Output
Output Device id  4  -  Altavoz/Auricular (Realtek High
Selected device is  Micrófono (Realtek High Definit
Yay!


In [27]:
p.get_default_host_api_info()

{'index': 0,
 'structVersion': 1,
 'type': 2,
 'name': 'MME',
 'deviceCount': 5,
 'defaultInputDevice': 1,
 'defaultOutputDevice': 4}